In [1]:
%%capture
%load_ext autoreload
%autoreload 2
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as pl
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, Norm_Iso2DGaussianModel, DoG_Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, Norm_Iso2DGaussianFitter, DoG_Iso2DGaussianFitter, CSS_Iso2DGaussianFitter

This notebook shows a simple example of prfpy use

# Creating stimulus object

In [2]:
screen_size_cm=40
screen_distance_cm=200
design_matrix=np.zeros((50,50,100))
design_matrix[:,20:30,40:50] = 1
TR=1
task_lengths=[100]
task_names=['test']
late_iso_dict={'test':np.arange(40)}
normalize_integral_dx=False

prf_stim = PRFStimulus2D(screen_size_cm=screen_size_cm,
                             screen_distance_cm=screen_distance_cm,
                             design_matrix=design_matrix,
                             TR=TR,
                             task_lengths=task_lengths,
                             task_names=task_names,
                             late_iso_dict=late_iso_dict,
                             normalize_integral_dx=normalize_integral_dx)

# Gaussian model fit

## Creating Gaussian model and fitter objects

In [3]:
#only needed if filtering predictions
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False

hrf=[1,0.4,0]

data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
#some kind of surround
data[:10,30:40] -= 2.5
data[:10,50:60] -= 2
data = np.roll(data,12)

normalize_RFs=False


gg = Iso2DGaussianModel(stimulus=prf_stim,
                          hrf=hrf,
                          filter_predictions=filter_predictions,
                          filter_type=filter_type,
                          filter_params=filter_params,
                          normalize_RFs=normalize_RFs)

gf = Iso2DGaussianFitter(data=data, model=gg, n_jobs=8)

## Gaussian grid fit

In [4]:
ecc_grid=np.linspace(0,10,10)
polar_grid=np.linspace(-np.pi,np.pi,10)
size_grid=np.linspace(1,10,10)
verbose=False
n_batches=8
fixed_grid_baseline=0
gauss_grid_bounds=[(0,1000)] #bound on prf amplitudes (only positive)

hrf_1_grid=np.linspace(0,10,10)
hrf_2_grid=np.linspace(0,0,1)

gf.grid_fit(ecc_grid=ecc_grid,
                polar_grid=polar_grid,
                size_grid=size_grid,
                verbose=verbose,
                n_batches=n_batches,
                fixed_grid_baseline=fixed_grid_baseline,
                grid_bounds=gauss_grid_bounds)#,
           #hrf_1_grid=hrf_1_grid,
           #hrf_2_grid=hrf_2_grid)

/Users/marcoaqil/prfpy/prfpy/model.py:315: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if current_hrf == 'direct':


In [5]:
gf.gridsearch_params[:,-1].max()

0.13661929965019226

## Gaussian Iterative Fit

In [6]:
rsq_threshold=0.01
verbose=True
gauss_bounds = [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0)]  # bold baseline
gauss_bounds += [(0.4,0.4),(0,0)] #hrf bounds. if want it fixed to some value, specify e.g. (4,4) (0,0)
constraints=None
xtol=1e-4
ftol=1e-4

gf.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                         bounds=gauss_bounds,
                         constraints=constraints,
                             xtol=xtol,
                             ftol=xtol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change i

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.05250D+02    |proj g|=  2.61480D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.40583D+02    |proj g|=  8.07177D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.63069D+02    |proj g|=  7.73070D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.64382D+02    |proj g|=  5.79803D-04
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate    1    f=  2.32162D+02    |proj g|=  1.73372D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1     10      1     0     0   1.734D-04   2.322D+02
  F =   232.16227339558313     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.87320D+02    |proj g|=  3.35376D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final proj


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.


...


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate    1    f=  2.80920D+02    |proj g|=  3.41061D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1     14      1     0     0   3.411D-04   2.809D+02
  F =   280.92043982462388     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  3.32782D+02    |proj g|=  2.21689D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final proj


   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.



At iterate    1    f=  2.85157D+02    |proj g|=  4.20641D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    4      1     30      1     0     0   4.206D-04   2.852D+02
  F =   285.15650895419947     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             



   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
[Parallel(n_jobs=8)]: Done  42 out of  42 | elapsed:    3.1s finished


In [7]:
gf.iterative_search_params

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         1.11325955e-02,  0.00000000e+00,  4.00000000e-01,
         0.00000000e+00,  2.37686568e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-4.61771251e-12, -4.61771251e-12,  1.00000000e+00,
         9.17289713e-03,  0.00000000e+00,  4.00000000e-01,
         0.00000000e+00,  1.21462793e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.692243

# DN model fit

## Create model and fitter objects

In [15]:
stimulus=prf_stim
filter_type='dc'
filter_params={"first_modes_to_remove":3,
                         "last_modes_to_remove_percent":0,
                         "window_length":50,
                         "polyorder":3,
                         "highpass":True,
                         "add_mean":True}

filter_predictions=False                                     
normalize_RFs=False

use_previous_gaussian_fitter_hrf=True #if true, will use hrf result from gauss fit at the grid stage instead of doing a grid fit for it

gg_norm = Norm_Iso2DGaussianModel(stimulus=prf_stim,
                                    hrf=hrf,
                                    filter_predictions=filter_predictions,
                                    filter_type=filter_type,
                                    filter_params=filter_params,                                       
                                    normalize_RFs=normalize_RFs)

gf_norm = Norm_Iso2DGaussianFitter(data=data,
                                   model=gg_norm,
                                   n_jobs=8,
                                   previous_gaussian_fitter=gf,
                                  use_previous_gaussian_fitter_hrf=use_previous_gaussian_fitter_hrf) 

## DN model grid fit

In [ ]:
norm_grid_bounds = [(0,1000),(0,1000)] #only prf amplitudes between 0 and 1000, only neural baseline values between 0 and 1000
num = 7
gf_norm.grid_fit(surround_amplitude_grid=np.linspace(0,10,num),
             surround_size_grid=np.linspace(1,10,num),
             neural_baseline_grid=np.linspace(0,10,num),
             surround_baseline_grid=np.linspace(1,10,num),
             verbose=verbose,
             n_batches=8,
             rsq_threshold=rsq_threshold,
             fixed_grid_baseline=fixed_grid_baseline,
             grid_bounds=norm_grid_bounds,
            hrf_1_grid=np.linspace(0,10,num),
            hrf_2_grid=np.linspace(0,0,1),
                ecc_grid=ecc_grid[:num],
                polar_grid=polar_grid[:num],
                size_grid=size_grid[:num])

Performing full grid for DN model


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/marcoaqil/anaconda3/envs/prfpy_2021/lib/python3.9/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change i

In [18]:
ecc_grid

array([ 0.        ,  1.11111111,  2.22222222,  3.33333333,  4.44444444,
        5.55555556,  6.66666667,  7.77777778,  8.88888889, 10.        ])

In [14]:
gf_norm.gridsearch_params

array([[-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         8.75593442e-03,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         4.00000000e-01,  0.00000000e+00,  1.41474260e-02],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         3.53425066e-03,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         4.00000000e-01,  0.00000000e+00,  1.80369266e-03],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         4.36300784e-03,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         4.00000000e-01,  0.00000000e+00,  1.35117187e-03],
       [-0.00000000e+00, -0.00000000e+00,  1.00000000e+00,
         1.16457846e-02,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         4.00000000e-01,  0.00000000e+00,  1.87865496e-02],
       [-0.00000000e+00, -0.00000000e+00,  1.0000000

## DN Iterative Fit

In [15]:
norm_bounds =  [(-1.5*10, 1.5*10),  # x
                (-1.5*10, 1.5*10),  # y
                (0.1, 1.5*5),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0),  # bold baseline
                (0, 1000),  # surround amplitude
                (0.1, 3*5),  # surround size
                (0, 1000),  # neural baseline
                (1e-6, 1000)]  # surround baseline
norm_bounds += [(0.4,0.4),(0,0)]
constraints_norm = None

gf_norm.iterative_fit(rsq_threshold=rsq_threshold, verbose=verbose,
                               bounds=norm_bounds,
                               constraints=constraints_norm,
                               xtol=xtol,
                               ftol=ftol)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.62009D+02    |proj g|=  1.20648D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.43961D+02    |proj g|=  1.11497D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  4.37982D+02    |proj g|=  9.60898D+01
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         2 variables are exactly at the bounds

At iterate    0    f=  2.76231D+02    |proj g|=  4.26782D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =     


At iterate    1    f=  2.68669D+02    |proj g|=  5.96422D+00

At iterate    1    f=  2.74587D+02    |proj g|=  1.84693D+01

At iterate    2    f=  2.68669D+02    |proj g|=  1.59094D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      2     14      4     0     1   1.591D+00   2.687D+02
  F =   268.66899209320627     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    2    f=  2.74586D+02    |proj g|=  5.59100D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = num

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.72400D+02    |proj g|=  6.14872D+01

At iterate    1    f=  3.12421D+02    |proj g|=  3.94691D+01

At iterate    2    f=  2.88952D+02    |proj g|=  2.14198D+01

At iterate    1    f=  3.02189D+02    |proj g|=  3.36089D+01

At iterate    1    f=  3.60066D+02    |proj g|=  3.79554D+01

At iterate    1    f=  3.16429D+02    |proj g|=  4.70260D+00

At iterate    2    f=  3.01646D+02    |proj g|=  1.49515D+01

At iterate    2    f=  3.35792D+02    |proj g|=  2.31368D+01

At iterate    1    f=  2.81200D+02    |proj g|=  2.53098D+01

At iterate    2    f=  3.16428D+02    |proj g|=  1.22634D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = nu

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            8     M =           10

At X0         2 variables are exactly at the bounds

At iterate    0    f=  2.56027D+02    |proj g|=  1.00000D+03

At iterate    1    f=  2.88574D+02    |proj g|=  2.46887D+01

At iterate    2    f=  2.88572D+02    |proj g|=  7.99376D+00

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
    8      2     14      5     0     1   7.994D+00   2.886D+02
  F =   288.57203658012594     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             

At iterate    1    f=  2.47159D+02    |proj g|=  1.30249D+01

At iter

[Parallel(n_jobs=8)]: Done  40 out of  40 | elapsed:    3.3s finished


# Crossvalidate

In [ ]:
test_data = 3*np.random.rand(50,100)+0.01*np.sum(design_matrix, axis=(0,1))-1.5
np.roll(test_data,6)

test_stimulus = prf_stim

single_hrf = False

gf.crossvalidate_fit(test_data=test_data,
                     test_stimulus=test_stimulus,
                     single_hrf=single_hrf)

gf_norm.crossvalidate_fit(test_data,
                        test_stimulus=test_stimulus,
                        single_hrf=single_hrf)